# Outline

Here we will train a fully-connected network to classify hand-written digits from images using the MNIST dataset

The network will take 28x28 pixel images and map them to digit categories {0, 1, 2, ..., 9}. 

In [0]:
# http://pytorch.org/

from os import path
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
accelerator = 'cu90' if path.exists('/opt/bin/nvidia-smi') else 'cpu'
!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.1-{platform}-linux_x86_64.whl torchvision
import torch
from torch import nn, optim
import torch.nn.functional as F
from torchvision import datasets, transforms

tcmalloc: large alloc 1073750016 bytes == 0x59cc0000 @  0x7f5824c941c4 0x46d6a4 0x5fcbcc 0x4c494d 0x54f3c4 0x553aaf 0x54e4c8 0x54f4f6 0x553aaf 0x54efc1 0x54f24d 0x553aaf 0x54efc1 0x54f24d 0x553aaf 0x54efc1 0x54f24d 0x551ee0 0x54e4c8 0x54f4f6 0x553aaf 0x54efc1 0x54f24d 0x551ee0 0x54efc1 0x54f24d 0x551ee0 0x54e4c8 0x54f4f6 0x553aaf 0x54e4c8


In [0]:
!pip freeze | grep torch

torch==0.4.1
torchvision==0.2.1


In [0]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving steering.csv to steering.csv
User uploaded file "steering.csv" with length 195642 bytes


In [0]:
!unzip Final_Four18-07-27-11_47_31.zip #unzip file

In [0]:
!cd Final_Four18-07-27-11_47_31/
!pwd

In [0]:
#!rm /content/Final_Four18-07-27-11_47_31/steering.csv
#!rm /content/steering*
!mv /content/steering.csv /content/Final_Four18-07-27-11_47_31

In [0]:
ls -l /content/Final_Four/DATA/Final_Four18-07-27-11\:47\:31/steering.csv

In [0]:
import os
print(os.getcwd())

/content


In [0]:
torch.cuda.set_device(0)
# On device 0
with torch.cuda.device(1):
    print("Inside device is 1")    
    # On device 1
print("Outside is still 0")
# On device 0

# Load the Dataset (MNIST)


We can use some PyTorch DataLoader utilities for this.  

In [0]:
!head /content/Final_Four18-07-27-11_47_31/steering.csv

In [0]:
import torch

from torch.utils.data.dataset import Dataset
from torch.utils.data import DataLoader
import numpy as np
import os
import random
from PIL import Image
import matplotlib.pyplot as plt
from torchvision import transforms
import csv


#from torch.utils.data.sampler import SubsetRandomSampler
#from torch.utils.data import random_split

from torch import randperm
from torch.utils import _accumulate

def random_split(dataset, lengths):
    """
    Randomly split a dataset into non-overlapping new datasets of given lengths.

    Arguments:
        dataset (Dataset): Dataset to be split
        lengths (sequence): lengths of splits to be produced
    """
    if sum(lengths) != len(dataset):
        raise ValueError("Sum of input lengths does not equal the length of the input dataset!")

    indices = randperm(sum(lengths))
    return [Subset(dataset, indices[offset - length:offset]) for offset, length in zip(_accumulate(lengths), lengths)]


  
#Type your own directory where you stored the data, fill in the blank!
FOLDER_DATASET = "/content/Final_Four18-07-27-11_47_31/"
# plt.ion()

class DriveData(Dataset):
    __xs = []
    __ys = []

    def __init__(self, folder_dataset, transform=None):
        self.transform = transform
        # Open and load text file including the whole training data
        with open(folder_dataset + "steering.csv") as f:
            reader = csv.reader(f)
            for line in reader:
                # Image path
                self.__xs.append(line[2])        
                # Steering wheel label
                self.__ys.append(np.float(line[0]))

    # Override to give PyTorch access to any image on the dataset
    def __getitem__(self, index):
        img = Image.open(self.__xs[index])
        img = img.convert('RGB')
        label = torch.from_numpy(np.asarray((self.__ys[index]-6000)/2000).reshape(1)).float()
        # random flip the images and reverse the steering command, DATA AUGMENTATION
        if random.randint(0, 1) == 0:
            img = flip(img)
            label = -label

        if self.transform is not None:
            img = self.transform(img)
        else:
            # Convert image and label to torch tensors
            img = np.transpose(np.asarray(img),(2,0,1)) #size [120,160,3]--->[3,120,160]
            img = torch.from_numpy(img/255.0) #size [3,120,160]
            
        img = img[:,60:120,:]

        return img, label

    # Override to give PyTorch size of dataset
    def __len__(self):
        return len(self.__xs)

# Please vist https://pytorch.org/docs/stable/torchvision/transforms.html to see different kinds of transformations
preprocessing = transforms.Compose([
   transforms.ToTensor(),
])


flip = transforms.Compose([
    transforms.RandomHorizontalFlip(p=1),
])


dset_train = DriveData(FOLDER_DATASET, transform=preprocessing)

train_size = int(0.8 * len(dset_train))
test_size = len(dset_train) - train_size
train_dataset, test_dataset = random_split(dset_train, [train_size, test_size])

#train_loader = DataLoader(dset_train, batch_size=5, shuffle=True, num_workers=1) #fill in the blank
#test_loader = DataLoader(dset_test, batch_size=5, shuffle=True, num_workers=1)



ImportError: ignored

In [0]:
!ls Final_Four18-07-27-11:47:31/steering.csv

ls: cannot access 'Final_Four18-07-27-11:47:31/steering.csv': No such file or directory


# Display some images

In [0]:
# matplotlib and stuff
import matplotlib.pyplot as plt
import numpy as np


def plt_style(c='k'):
    """
    Set plotting style for bright (``c = 'w'``) or dark (``c = 'k'``) backgrounds

    :param c: color, can be set to ``'w'`` or ``'k'`` (which is the default)
    :type c: str
    """
    import matplotlib as mpl
    from matplotlib import rc

    # Reset previous configuration
    mpl.rcParams.update(mpl.rcParamsDefault)
    #%matplotlib inline  # not from script
    get_ipython().run_line_magic('matplotlib', 'inline')

    # configuration for bright background
    if c == 'w':
        plt.style.use('bmh')

    # configurations for dark background
    if c == 'k':
        plt.style.use(['dark_background', 'bmh'])

    # remove background colour, set figure size
    rc('figure', figsize=(16, 8), max_open_warning=False)
    rc('axes', facecolor='none')
    rc('nbagg', transparent=False)


def plt_interactive(c='k'):
    from matplotlib import rc
    import matplotlib as mpl
    mpl.rcParams.update(mpl.rcParamsDefault)
    get_ipython().run_line_magic('matplotlib', 'notebook')
    plt.rc('figure', figsize=(160, 60), facecolor=c)
    rc('nbagg', transparent=False)
    # configuration for bright background
    if c == 'w':
        plt.style.use('bmh')

    # configurations for dark background
    if c == 'k':
        plt.style.use(['dark_background', 'bmh'])
    rc('axes', facecolor='none')

plt_style()

In [0]:
#!pip install Pillow==4.0.0
#!pip install PIL
#!pip install image
from PIL import Image

In [0]:
for i in range(10):
    imgs, steering_angle = next(iter(train_loader))
    #print('Batch shape:',imgs.size())
    #print(steering_angle)

    #plt.imshow(np.transpose(imgs.numpy()[0,:,:,:],(1,2,0)))
    #plt.show()
    #plt.imshow(np.transpose(imgs.numpy()[-1,:,:,:],(1,2,0)))
    #plt.show()

# Create the model class

In [0]:
class ConvNet(nn.Module):
  def __init__(self, num_classes):
    super(ConvNet, self).__init__()
    self.layer1 = nn.Sequential(
      nn.Conv2d(3, 9, kernel_size=5, stride=1, padding=0),
      nn.ReLU(),
      nn.MaxPool2d(kernel_size=2, stride=2))
    self.layer2 = nn.Sequential(
      nn.Conv2d(9, 27, kernel_size=5, stride=1, padding=0),
      nn.ReLU(),
      nn.MaxPool2d(kernel_size=2, stride=2))
    self.layer3 = nn.Sequential(
      nn.Conv2d(27, 71, kernel_size=4, stride=1, padding=0),
      nn.ReLU(),
      nn.MaxPool2d(kernel_size=2, stride=2))
    self.layer4 = nn.Sequential(
      nn.Conv2d(71, 140, kernel_size=3, stride=1, padding=0),
      nn.ReLU(),
      nn.MaxPool2d(kernel_size=2, stride=2))
    #self.layer5 = nn.Sequential(
     #  nn.Conv2d(140,160, kernel_size=3, stride=1, padding=0),
     #  nn.ReLU(),
     #  nn.MaxPool2d(kernel_size=2, stride=2))


    self.fc1 = nn.Sequential(
        nn.Linear(1*7*140, 100),
        nn.ReLU())
    self.fc2 = nn.Sequential(
        nn.Linear(100, 50),
        nn.ReLU())
    self.fc3 = nn.Sequential(
        nn.Linear(50, 10),
        nn.ReLU())
    self.fc4 = nn.Sequential(
        nn.Linear(10, num_classes))


  def forward(self, x):
    out = self.layer1(x)
    out = self.layer2(out)
    out = self.layer3(out)
    out = self.layer4(out)
    #out = self.layer5(out)
    out = out.reshape(out.size(0), -1)
    out = self.fc1(out)
    out = self.fc2(out)
    out = self.fc3(out)
    out = self.fc4(out)

    return out

# Define Training Loop

* Loop batches of samples in the training set
* Run each batch through the model (forward pass)
* Compute the loss
* Compute the gradients with respect to model parameters (backward pass)
* Update the parameters

In [0]:
# Define training loop

def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        # Move tensors to the configured device
        data = data.to(device)
        target = target.to(device)
        ########### Forward pass #############
        y = model(data)
        ##########calculate the loss##########

        current_loss = F.mse_loss(y, target)
        
        
        ###### backpropagation and optimize###

        model.zero_grad()
        current_loss.backward()
        optimizer.step()
        
        #######################################
        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100 * batch_idx / len(train_loader), current_loss.item()))
    return(current_loss)


In [0]:
import plotly

# Define Testing Loop

* Loop over batches of samples in the testing set
* Run each batch through the model (forward pass)
* Compute the loss and accuracy
* Do not compute gradients or update model parameters 
* We are saving the testing data to evaluate how the model is doing on data it has not been trained on

In [0]:
def test():
    model.eval()
    test_loss = 0
    correct = 0
    
    # In test phase, we don't need to compute gradients (for memory efficiency)
    with torch.no_grad():
      for data, target in test_loader:
          # Move tensors to the configured device
          data = data.to(device)
          target = target.to(device)
          #############Forward pass#############
          output = model(data)
          output = output.double
          ######################################
          test_loss += F.mse_loss(output, target, size_average=False).item() # sum up batch loss                                                               
          pred = output.data.max(1, keepdim=True)[1] # get the index of the max log-probability                                                                 
          correct += pred.eq(target.data.view_as(pred)).cpu().sum()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100 * correct / len(test_loader.dataset)))

# Initialize the Model and Optimizer

In [0]:
# Device configuration
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# Hyper parameters
num_epochs = 10
learning_rate = 0.001


#Initiate the model object using the class we've already defined
num_classes = 1
model = ConvNet(num_classes)


#Move the model object to the Device
model = model.to(device)

#choose your desired optimizer##
optimizer = optim.SGD(model.parameters(), learning_rate, momentum = 0.5) 


In [0]:
!pip freeze

In [0]:
!python3 -c 'import torch; print(torch); print(torch.__path__)'

<module 'torch' from '/usr/local/lib/python3.6/dist-packages/torch/__init__.py'>
['/usr/local/lib/python3.6/dist-packages/torch']


# Train the Model

* We will only train for a few epochs here
* Normally we would train for longer
* Depending on the dataset and model size, this can take days or weeks

In [0]:
#Use a for loop to call train function and testing function
#In this way, the model can be trained and tested in few epoches
#######################################################
import matplotlib.pyplot as plt
num_epochs = 10
epochs = []
losses = []

for i in range(num_epochs):
  #train(i)
  loss = train(i)
  print("loss " , loss)
  print(i)
  epochs.append(i)
  losses.append(loss)
  
  test()
plt.plot(epochs, losses)           
#######################################################

Train Epoch: 0 [0/7246 (0%)]	Loss: 0.412578
Train Epoch: 0 [500/7246 (7%)]	Loss: 0.061782
Train Epoch: 0 [1000/7246 (14%)]	Loss: 0.266654
Train Epoch: 0 [1500/7246 (21%)]	Loss: 0.571561
Train Epoch: 0 [2000/7246 (28%)]	Loss: 0.379372
Train Epoch: 0 [2500/7246 (34%)]	Loss: 0.198860
Train Epoch: 0 [3000/7246 (41%)]	Loss: 0.268807
Train Epoch: 0 [3500/7246 (48%)]	Loss: 0.300486
Train Epoch: 0 [4000/7246 (55%)]	Loss: 0.497002
Train Epoch: 0 [4500/7246 (62%)]	Loss: 0.241866
Train Epoch: 0 [5000/7246 (69%)]	Loss: 0.231271
Train Epoch: 0 [5500/7246 (76%)]	Loss: 0.301036
Train Epoch: 0 [6000/7246 (83%)]	Loss: 0.178813
Train Epoch: 0 [6500/7246 (90%)]	Loss: 0.378770
Train Epoch: 0 [7000/7246 (97%)]	Loss: 0.079951
loss  tensor(1.2809, device='cuda:0')
0


TypeError: ignored

In [0]:
-

# Now show some model predictions on testing data

* We will show an image from the testing set, and the probabilities the model assigns to each class

In [0]:
# show an image and the predicted probabilities                                                                                                               
def display(i):
    plt.figure(i + 1)
    image, _ = test_loader.dataset.__getitem__(i)
    image = image.reshape(1, 1, 28, 28).to(device)
    output = model(image)
    prob = F.softmax(output).squeeze().data
    plt.subplot(121)
    plt.imshow(image.squeeze().cpu().numpy())
    plt.subplot(122)
    plt.bar(range(10), prob.cpu().numpy())
    plt.xlabel('predicted class probabilities')
    plt.xticks(range(10))
    plt.ylim([0, 1])

for i in range(10):
    display(i)

# Things to try out

* Try different numbers of layers and hidden units
* Try different non-linearities (tanh, sigmoid)
* Try on other datasets (CIFAR 10)

In [0]:
class ConvNet(nn.Module):
  def __init__(self, num_classes):
    super(ConvNet, self).__init__()
    self.layer1 = nn.Sequential(
    nn.Conv2d(1, 16, kernel_size=5, stride=1, padding=0),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=2, stride=2))
    self.layer2 = nn.Sequential(
    nn.Conv2d(16, 32, kernel_size=5, stride=1, padding=0),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=2, stride=2))
    self.fc = nn.Linear(4*4*32, num_classes)
    
  def forward(self, x):
    out = self.layer1(x)
    out = self.layer2(out)
    out = out.reshape(out.size(0), -1)
    out = self.fc(out)
    return out
  
